In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import numpy as np
from cycler import cycler

In [13]:
# Read the data
msv_data = pd.read_csv("../../gt_preprocessed_data/gt_msv_stitched/3_gt_msv_stitched_compute.csv")
rsv_data = pd.read_csv("../../gt_preprocessed_data/gt_rsv_stitched/3_gt_rescaled_rsv.csv")

In [14]:
# Convert date column to datetime
msv_data['date'] = pd.to_datetime(msv_data['date'])
rsv_data['date'] = pd.to_datetime(rsv_data['date'])

In [16]:
# Define events with their dates, colors, and vertical text positions
events = {
    "2020-03-17": ("First ECQ ongoing; PH under 'state of calamity'", "#FF1493", 1.0),
    "2020-03-25": ("Bayanihan Act signed", "#FF0505", 1.1),
    "2020-04-07": ("ECQ in MM extended to Apr 30", "#FF1493", 1.0),
    "2020-04-24": ("ECQ in MM extended to May 15", "#FF1493", 1.1),
    "2020-05-12": ("IATF puts MECQ in MM", "#FF1493", 1.0),
    "2020-05-26": ("MM mayors agreed GCQ", "#FF1493", 1.1),
    "2020-06-01": ("GCQ starts in MM", "#FF1493", 1.0),
    "2020-08-02": ("100k cases surpassed in PH", "#000000", 1.0),
    "2020-08-04": ("MECQ imposed again in MM", "#FF1493", 1.0),
    "2020-08-19": ("PhilHealth scandal", "#FF0505", 1.1),
    "2020-09-18": ("Bayanihan 2 Act signed", "#FF0505", 1.0),
    "2020-09-28": ("All PH provinces infected", "#000000", 1.1),
    "2020-12-19": ("Alpha in UK", "#000000", 1.0),
    "2021-01-09": ("DOH monitors Beta/Delta overseas", "#000000", 1.0),
    "2021-01-13": ("Alpha in PH", "#000000", 2.0),
    "2021-01-14": ("Pfizer approved", "#FFD700", 3.0),
    "2021-01-28": ("AstraZeneca approved", "#FFD700", 1.0),
    "2021-03-02": ("Beta in Pasay", "#000000", 1.1),
    "2021-03-12": ("Gamma in PH", "#000000", 1.0)
}

In [17]:
def add_event_lines(ax, ylim):
    """Add vertical lines for events with annotations"""
    # Store existing lines to adjust zorder later
    existing_lines = ax.get_lines()
    
    # Set all data lines to lower z-order
    for line in existing_lines:
        line.set_zorder(1)
        
    # Draw event lines with increased width and higher z-order
    for date, (label, color, pos) in events.items():
        event_date = pd.to_datetime(date)
        ax.axvline(x=event_date, 
                  color=color, 
                  linestyle='--', 
                  alpha=0.8,
                  linewidth=2.5,
                  zorder=5)
        
        # Special handling for overlapping text
        if label == "Pfizer approved":
            x_offset = 14 
        elif label == "MECQ imposed again in MM":
            x_offset = 14
        else:
            x_offset = 5  # Default x-offset for other labels
            
        # Add text with rotation and offset
        ax.annotate(label, 
                   xy=(event_date, ylim[1]),
                   xytext=(x_offset, 10),  # Using the dynamic x_offset
                   textcoords='offset points',
                   rotation=45,
                   ha='left',
                   va='bottom',
                   fontsize=8,
                   zorder=6)

In [18]:
def create_individual_graphs(data, data_type):
    """Create individual graphs for each tag"""
    tags = [col for col in data.columns if col != 'date']
    
    for tag in tags:
        plt.figure(figsize=(15, 8))
        plt.plot(data['date'], data[tag], label=tag, linewidth=2)
        
        # Add event lines
        ylim = plt.ylim()
        add_event_lines(plt.gca(), ylim)
        
        plt.xlabel('Date')
        plt.ylabel('Value')
        
        # Move title to bottom
        plt.figtext(0.5, 0.02, f'{data_type} Data Processing - {tag} Google Trends Values',
                   ha='center', va='center', fontsize=12)
        
        plt.tight_layout()
        plt.subplots_adjust(bottom=0.1)  # Make room for bottom title
        plt.savefig(f'{data_type}_{tag}_timeseries.png')
        plt.close()

In [21]:
# Define groups
MSV_GROUPS = {
    "MSVSymptoms-0.6": ["cough", "fever", "sipon"],
    "MSVCOVIDProtocols1-0.6": ["ecq", "Quarantine"],
    "MSVSymptoms&HealthProtocols-0.6": ["flu", "headache", "lagnat", "rashes", "ubo", 
                                        "face shield", "Frontliners", "masks", "social distancing"],
    "MSVSymptoms-0.5": ["flu", "cough", "fever", "sipon", "ubo"],
    "MSVCOVIDProtocols1-0.5": ["Quarantine", "Frontliners", "ecq"],
    "MSVFaceWearing&Others-0.5": ["masks", "face shield", "headache", "lagnat", "rashes", "social distancing"]
}

RSV_GROUPS = {
    "RSVSymptoms&HealthProtocols1-0.6": ["flu", "cough", "fever", "Quarantine", "work from home"],
    "RSVSymptoms&HealthProtocols2-0.6": ["face shield", "headache", "lagnat", "rashes", "sipon", 
                                        "ubo", "Frontliners", "masks", "social distancing"],
    "RSVSymptoms&COVIDProtocols-0.5": ["social distancing", "work from home", "Quarantine", 
                                     "headache", "fever", "cough", "flu"],
    "RSVFaceWearing&Others-0.5": ["face shield", "masks", "Frontliners", "ubo", "sipon", "rashes", "lagnat"]
}

In [22]:
# Generate all visualizations
create_individual_graphs(msv_data, 'MSV')
create_individual_graphs(rsv_data, 'RSV')